In [1]:
import pymysql, os, copy, json, time
import pandas as pd
import numpy as np
from scipy.stats import mode

In [2]:
def _call_db_info(db_name):
    return pymysql.connect(
        host = '10.94.77.9',
        port= 13306,
        user = 'dxbot1',
        password='ensol2020!',
        db = db_name,
        charset = 'utf8')

In [3]:
conn = _call_db_info('gscm_e61d')
curs = conn.cursor()
curs.execute("select * from dt_table;") 
dt_table = pd.DataFrame(curs.fetchall())
conn.commit()
conn.close()
dt_table.columns = ['WW']    

In [4]:
start = 202001

## Inventory Result

In [9]:
tmp = pd.read_csv('./data1/Inven_3xxC.csv')
tmp['Inven Week']

0      201901
1      201902
2      201903
3      201904
4      201905
        ...  
116    202112
117    202113
118    202114
119    202115
120    202116
Name: Inven Week, Length: 121, dtype: int64

In [6]:
insert_inven_sql = "insert into inven_rslt values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"#.format(*c)
tmp = pd.read_csv('./data1/Inven_3xxC.csv')
ids = tmp.index[tmp['Inven Week'] == start].tolist()[0]

conn = _call_db_info('gscm_e61d')
curs = conn.cursor()
for i in range(ids, tmp.shape[0]):
    tmp_s = dt_table.index[dt_table['WW']==tmp['Inven Week'].loc[i]].tolist()[0]    
    tmp_e = tmp_s - 8    
    ww_candidate = dt_table.loc[tmp_e:tmp_s].reset_index(drop=True)  
    tmp_join = pd.merge(ww_candidate, tmp, left_on='WW', right_on='Inven Week')
    tmp_join = tmp_join.sort_values('WW', ascending=False)      
    inven_list = list(tmp_join[tmp_join['WW']==tmp['Inven Week'].loc[i]][['Inven Week', 'Date']].values.reshape(-1))
    inven_list.extend(tmp_join['재고수량'].values.tolist())
    curs.execute(insert_inven_sql, inven_list) 
conn.commit()
conn.close()    

## Product Result

In [ ]:
insert_prod_sql = "insert into prod_rslt values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
tmp = pd.read_csv('./data1/02.Detail_생산계획 준수율.csv')
ids = tmp.index[tmp['Plan Week'] == start].tolist()[0]

conn = _call_db_info('gscm_e61d')
curs = conn.cursor()
for i in range(ids, tmp.shape[0]):
    tmp_s = dt_table.index[dt_table['WW']==tmp['Plan Week'].loc[i]].tolist()[0]    
    tmp_e = tmp_s - 12
    ww_candidate = dt_table.loc[tmp_e:tmp_s].reset_index(drop=True)  
    tmp_join = pd.merge(ww_candidate, tmp, how='left',  left_on='WW', right_on='Plan Week')       
    tmp_join = tmp_join.sort_values('WW', ascending=False)
    tmp_join = tmp_join.fillna(0)    
    prod_list = list(tmp_join[tmp_join['WW']==tmp['Plan Week'].loc[i]][['Plan Week', 'Basic Date']].values.reshape(-1))
    prod_list.append(tmp_join['Result Qty'])    
    curs.execute(insert_prod_sql, prod_list) 
conn.commit()
conn.close()      

## Product Plan

In [23]:
insert_prod_sql = "insert into prod_plan values (%s, %s, %s)"
tmp = pd.read_csv('./data1/DynamicResult.csv')
current = max(tmp['WW'])

tmp_s = dt_table.index[dt_table['WW'] == start].tolist()[0]
tmp_e = dt_table.index[dt_table['WW'] == current].tolist()[0]

#conn = _call_db_info('gscm_e61d')
#curs = conn.cursor()
for i in range(tmp_s, (tmp_e+1)):    
    tmp_ = tmp[tmp['WW'] == dt_table['WW'].loc[i]].reset_index(drop=True)
    tmp_ = tmp_.sort_values('tWW')
    tmp_plan = list(map(lambda x : [str(tmp_['tWW'].values[x]), str(tmp_['QTY'].values[x])], range(0, tmp_.shape[0])))
    tmp_plan_join = str('/'.join(list(map(lambda x : ':'.join(tmp_plan[x]), range(0, tmp_.shape[0])))))
    prod_plan = list(tmp_[['WW','Date']].loc[0].tolist())
    prod_plan.append(tmp_plan_join)
#    curs.execute(insert_prod_sql, prod_plan) 
#conn.commit()
#conn.close() 
print(prod_plan)

[202118, '2021-05-03', '202119:48000.0/202120:208000.0/202121:128000.0/202122:176000.0/202123:208000.0/202124:208000.0/202125:422500.0/202126:422500.0/202127:198000.0/202128:214500.0/202129:214500.0/202130:214500.0/202131:198000.0/202132:214500.0/202133:214500.0/202134:214500.0/202135:214500.0/202136:402000.0/202137:435500.0/202138:214500.0/202139:214500.0/202140:198000.0/202141:214500.0/202142:214500.0/202143:214500.0/202144:198000.0/202145:214500.0/202146:214500.0/202147:214500.0/202148:214500.0/202149:181500.0/202150:214500.0/202151:234500.0/202152:435500.0/202201:368500.0/202202:401500.0/202203:214500.0/202204:214500.0/202205:214500.0/202206:181500.0/202207:214500.0/202208:401500.0/202209:435500.0/202210:401500.0/202211:435500.0/202212:214500.0/202213:214500.0/202214:115500.0/202215:214500.0/202216:214500.0/202217:214500.0/202218:3799643.053583692']


## Product Prev

In [41]:
insert_prod_sql = "insert into prod_prev values (%s, %s, %s)"
tmp = pd.read_csv('./data1/DynamicResult.csv')
current = max(tmp['WW'])

tmp_s = dt_table.index[dt_table['WW'] == start].tolist()[0]
tmp_e = dt_table.index[dt_table['WW'] == current].tolist()[0]

conn = _call_db_info('gscm_e61d')
curs = conn.cursor()
for i in range(tmp_s, (tmp_e+1)):#(tmp_s+1)):    #
    tmp_ = tmp[tmp['tWW'] == dt_table['WW'].loc[i]].reset_index(drop=True)
    tmp_ = tmp_.sort_values('WW')[::-1].reset_index(drop=True)
    tmp_prev = list(map(lambda x : [str(tmp_['WW'].values[x]), str(tmp_['QTY'].values[x])], range(1, tmp_.shape[0])))
    tmp_prev_join = str('/'.join(list(map(lambda x : ':'.join(tmp_prev[x]), range(0, len(tmp_prev))))))
    prod_prev = list(tmp_[['tWW','Date']].loc[0].tolist())
    prod_prev.append(tmp_prev_join)
    #print(i, prod_prev)
    curs.execute(insert_prod_sql, prod_prev) 
conn.commit()
conn.close() 

## Sale Plan

In [7]:
insert_sale_sql = "insert into sale_plan values (%s, %s, %s)"
f_list = os.listdir('./data1/')
f_list.sort()
sale_file = [i for i, v in enumerate(f_list) if 'Fcst' in v.split(' ')]
start = 202001
current = 202116

tmp_s = dt_table.index[dt_table['WW']==start].tolist()[0]#-12
tmp_e = dt_table.index[dt_table['WW']==current].tolist()[0]

conn = _call_db_info('gscm_e61d')
curs = conn.cursor()
for i in range(tmp_s, (tmp_e+1)):
    tmp_ww = dt_table['WW'].loc[i]
    cnt = 0
    for j in range(0, len(sale_file)):#tmp_s, tmp_e):    
        tmp = pd.read_csv('./data1/'+f_list[sale_file[j]])
        if tmp_ww in tmp['WW'].values and str(tmp_ww) in tmp.columns:
            #if tmp_ww in tmp['WW'] and tmp_ww in tmp.columns
            row_id = tmp.index[tmp['WW'] == tmp_ww][0]
            col_id = list(tmp.columns).index(str(tmp_ww))        
            if cnt == 0 :
                tmp_ww_plan = tmp.iloc[row_id, col_id:]
            else:
                tmp_ww_plan = pd.concat([tmp_ww_plan, tmp.iloc[row_id, col_id:]])
            cnt+=1
            tmp_date = tmp[tmp['WW'] == tmp_ww]['Date'].values[0]
    tmp_ww_plan = tmp_ww_plan.reset_index(drop=False).drop_duplicates().sort_values('index')
    tmp_ww_group = tmp_ww_plan.groupby('index')
    tmp_ww_rslt = pd.DataFrame(tmp_ww_group.max()).reset_index(drop=False)        
    tmp_ww_rslt.columns=['ww', 'qty']
    tmp_plan = list(map(lambda x : [str(tmp_ww_rslt['ww'].values[x]), str(tmp_ww_rslt['qty'].values[x])], range(0, tmp_ww_rslt.shape[0])))
    tmp_plan_join = str('/'.join(list(map(lambda x : ':'.join(tmp_plan[x]), range(0, tmp_ww_rslt.shape[0])))))
    
    sale_plan = list([tmp_ww,tmp_date])#list(tmp[['WW', 'Date']].loc[row_id].values.reshape(-1))
    sale_plan.append(tmp_plan_join)
    curs.execute(insert_sale_sql, sale_plan) 
conn.commit()
conn.close()

## Sale Result

In [ ]:
insert_sale_sql = "insert into sale_rslt values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
f_list = os.listdir('./data1/')
f_list.sort()
sale_file = [i for i, v in enumerate(f_list) if 'Fcst' in v.split(' ')]
start = 202001
current = 202116

tmp_s = dt_table.index[dt_table['WW']==start].tolist()[0]
tmp_e = dt_table.index[dt_table['WW']==current].tolist()[0]
tmp_rslt = []
for i in range((tmp_s-12), (tmp_e+1)):
    tmp_ww = dt_table['WW'].loc[i]    
    tmp_ww_rslt = []
    for j in range(0, len(sale_file)):
        tmp = pd.read_csv('./data1/'+f_list[sale_file[j]])
        if str(tmp_ww) in tmp.columns:
            col_id = list(tmp.columns).index(str(tmp_ww))     
            row_id = tmp.index[tmp['WW'] > tmp_ww].values.tolist()  
            tmp_ww_rslt.extend(tmp.iloc[row_id, col_id])
            if any(tmp_ww == tmp['WW']):
                tmp_date = tmp[tmp['WW'] == tmp_ww]['Date'].values[0]
    if np.sum(tmp_ww_rslt) > 0 :
        tmp_mode = mode(np.array(tmp_ww_rslt)[np.array(tmp_ww_rslt) > 0]).mode[0]            
    else:
        tmp_mode = 0
    tmp_rslt.append([tmp_ww, tmp_date, tmp_mode])
tmp_rslt = pd.DataFrame(tmp_rslt)

conn = _call_db_info('gscm_e61d')
curs = conn.cursor()
for i in range(tmp_s, (tmp_e+1)):
    tmp_ww = dt_table['WW'].loc[i]
    tmp_ind = tmp_rslt[tmp_rslt[0] == tmp_ww].index[0]
    tmp_sale_rslt = tmp_rslt.iloc[(tmp_ind-12):tmp_ind, 2][::-1].values.tolist()
    sale_rslt = tmp_rslt.iloc[tmp_ind, :].values.tolist()
    sale_rslt.extend(tmp_sale_rslt)    
    curs.execute(insert_sale_sql, sale_rslt) 
conn.commit()
conn.close()

## Sale Prev

In [41]:
insert_sale_sql = "insert into sale_prev values (%s, %s, %s)"
f_list = os.listdir('./data1/')
f_list.sort()
sale_file = [i for i, v in enumerate(f_list) if 'Fcst' in v.split(' ')]
start = 202001
current = 202116
tmp_s = dt_table.index[dt_table['WW']==start].tolist()[0]#-12
tmp_e = dt_table.index[dt_table['WW']==current].tolist()[0]
print(tmp_s, tmp_e)

conn = _call_db_info('gscm_e61d')
curs = conn.cursor()

for i in range(tmp_e, (tmp_e+1)):
    tmp_ww = dt_table['WW'].loc[i]    
    cnt = 0
    for j in range(0, len(sale_file)):#tmp_s, tmp_e):    
        tmp = pd.read_csv('./data1/'+f_list[sale_file[j]])    
        if tmp_ww in tmp['WW'].values and str(tmp_ww) in tmp.columns:        
            row_id = tmp.index[tmp['WW'].values < tmp_ww].tolist()
            col_id = list(tmp.columns).index(str(tmp_ww))  
            tmp_ = tmp[['WW','Date']].copy()#.iloc[row_id]#,[0,1,col_id]]
            tmp_['tmp_prev'] = tmp.iloc[:,col_id]
            if any(tmp_ww == tmp['WW']):
                tmp_date = tmp[tmp['WW'] == tmp_ww]['Date'].values[0]
            if cnt == 0 :
                tmp_prev_plan = tmp_.iloc[row_id, :]
            else:
                tmp_prev_plan = pd.concat([tmp_prev_plan, tmp_.iloc[row_id, :]])
            cnt+=1
    tmp_prev_plan_group = tmp_prev_plan.groupby(['WW', 'Date'])
    #tmp_prev_plan_ = tmp_prev_plan.drop_duplicates().sort_values('WW', ascending=False).reset_index(drop=True)
    tmp_prev_plan_ = pd.DataFrame(tmp_prev_plan_group.max()).sort_values('WW', ascending=False).reset_index(drop=False)                     
    tmp_plan = list(map(lambda x : [str(tmp_prev_plan_['WW'].values[x]), str(tmp_prev_plan_['tmp_prev'].values[x])], range(0, tmp_prev_plan_.shape[0])))
    tmp_plan_join = str('/'.join(list(map(lambda x : ':'.join(tmp_plan[x]), range(0, len(tmp_plan))))))
    sale_prev = list([tmp_ww, tmp_date])#list(tmp[['WW', 'Date']].loc[row_id].values.reshape(-1))
    sale_prev.append(tmp_plan_join)
    
    curs.execute(insert_sale_sql, sale_prev) 
conn.commit()
conn.close()

52 120
